# Original discussion about `gene:line_number`

really?  Another way would be to compute a priori the line number for each gene, save as a Python dictionary in SoS global variable,  then use sed to get to that specific line when you have a gene name -- does this big picture idea make sense? (edited)
32 replies

Jiarun  3 days ago
really?
Actually my command was zless *.gz | grep ENSG... > some.tmp.file, which took so long. However, I may not rule out the possibility of RCC causing that slow.

Jiarun  3 days ago
Another way would be to compute a priori the line number for each gene
So the key is to run some code before the pipeline, which runs once per gene. (Let me try to think of a way first perhaps) (edited)

gaow  3 days ago
So the key is to run some code before the pipeline,
yes, or rather, as a preprocessing step of the pipeline!
which runs once per gene.
No it runs once per y or z file

gaow  3 days ago
which is 2 * number of tissues total?

gaow  3 days ago
all you need is the first column of each file, the gene name, and then get their line number

gaow  3 days ago
i suggest you first learn to use filename control and derivation via SoS vairables; otherwise it will be a mess.

Jiarun  3 days ago
yes, or rather, as a preprocessing step of the pipeline!
That’s a good idea!

Jiarun  3 days ago
No it runs once per y or z file
I see you mean the grep ENSG... step. And the step of reading from files the expression of all 20000 genes can also be treated as preprocessing, right? I mean, in a separate preprocessing section in the SoS Notebook. (Please forgive my unfamiliarity with SoS) (edited)

gaow  3 days ago
the step of reading from files the expression of all 20000 genes can also be treated as preprocessing, right?
you will not have this step if you get a list of gene:line_number

gaow  3 days ago
because you will sed that line number for the gene instead of grep

Jiarun  3 days ago
Oh. You mean using bash instead of R (or Python)! (edited)

gaow  3 days ago
so you always get one line of expression for the gene of interest. this is just a faster alternateive to grep.

gaow  3 days ago
Well good point, or you can figure out how to read a specific line in R rather than the whole file that will be even better!

gaow  3 days ago
maybe you google it and see. if so, then in R you first get the line number of interest, then use R to read that line. I believe at least you can "read from standard output stream in R" so you acn do read.xxx('sed ... ') in R

gaow  3 days ago
(google how to read standard stream in R if you cannot find how to read a line in R)

Jiarun  2 days ago
OK. I will search. But I suspect that directly reading all the 49 gene expression files as preprocessing and extracting gene expression for each gene by subsetting the huge variable of expression of all genes based on the prespecified gene:line_number would be faster than reading a specific line in R, mainly for my time of development, perhaps even in terms of running time. (edited)

Jiarun  16 hours ago
Gao, I think this problem is actually intrinsic. I mean, no matter which approach we use, the essential step is to search the location of a gene in a file with tissue-specific gene expressions, which takes some 1s in either bash grep or R grepl() or Python file IO, since the algorithm doesn’t seem to be able to improve.

Jiarun  16 hours ago
Oh wait, but even if it takes 1min to generate RDS for one gene, the whole procedure will still finish within 2h, which I can definitely bear. So is that OK? (edited)

gaow  12 hours ago
Did you try how long it takes if you know the line number ? No grep is involved ...

gaow  12 hours ago
Would be helpful if you read my earlier suggestions again in case you didn't fully understand it?

Jiarun  12 hours ago
OK. I will read them again.

Jiarun  12 hours ago
I didn’t find anything new …

Jiarun  12 hours ago
But grep is actually included in the process of generating gene:line_number, I mean, at least each file has to be traversed. (edited)

gaow  12 hours ago
@Jiarun can you just read the first column (in Python pandas) the gene names, then assign line numbers to them, for each tissue?

gaow  12 hours ago
once you have gene:lin_number you can sed or other methods to read a line, which should be fast -- is this part clear?

Jiarun  12 hours ago
Oh! I find the message above! I was too careless!

Jiarun  12 hours ago
(Actually when I calculated the complexity to see why it takes ~1min, I saw that traversing the whole files is highly unnecessary, but I just failed to think of extracting gene names first!


Jiarun  12 hours ago
I have tested reading specific line by sed

# Crystallized plan for generating y with the main step `gene:line_number`

1) extract the column of ensembl_gene_id in the 49 expression files by `pandas`

2) assign `gene:line_number` for each gene in each tissue based on the extracted column just now

3) for each gene, extract the row of expression of the gene with sample names in each tissue using `sed`

4) that's it!

In [25]:
# preprocessing: prepare gene expressions, gene TSS.
[preprocessing]
depends: R_library("biomaRt"), R_library("data.table")

bash: expand = '${ }'
    # `ensembl_gene_id.txt`
    cd /project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_expression_matrices
    for file in ./*.gz; do zless "$file" | grep -oh "ENSG\w*\.\w*" >> /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/ensembl_gene_id_original.txt ; done
    cd /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/
    sort -u ensembl_gene_id_original.txt > ensembl_gene_id.txt
    rm ensembl_gene_id_original.txt
    
    # `samplenames_X.txt`
    zless /project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/genotypes/WGS/variant_calls/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.vcf.gz | sed '3386q;d' > /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/samplenames_X.txt

python3: expand = '${ }'
    # `gene:line_number`
    
    filenames_y = glob.glob("/Users/cjr/Documents/Enloc/eQTL/GTEx_Analysis_v8_eQTL_expression_matrices/*.gz")
    filenames_y.sort()
    gene_linenumber = {} # Now gene_linenumber is a list of 49 dict, each of which has 20000 items. But if gene_linenumber is a dict of 20000 dict, each of which has 49 items, then it can be saved into 20000 files!
    fields = ['gene_id', ]
    for j in range(len(ensembl_gene_id)): # construct ~20000 dict, each of which has 49 items
        gene_linenumber[ensembl_gene_id[j]] = {}

    for i in range(len(filenames_y)): # construct each of the 49 `gene:line_number`
        gene_id = pd.read_csv(filenames_y[i], sep='\t', skipinitialspace=True, usecols=fields)
        for j in range(gene_id.shape[0]): # go over lines
            gene_linenumber[gene_id['gene_id'].tolist()[j]][i] = j + 2
            print('%s at line %d in file %d' % (gene_id['gene_id'].tolist()[j], j + 2, i))
    
    import csv
    for j in range(len(gene_linenumber)): # write ~20000 files
        with open(('/Users/cjr/Documents/Enloc/eQTL/gene-line_number/%s.csv' % ensembl_gene_id[j]), 'w') as f:
            writer = csv.writer(f)
            for key, value in gene_linenumber[ensembl_gene_id[j]].items():
                writer.writerow([key, value])
    
    # Now gene_linenumber is a list of 49 dict, each of which has 20000 items. But if gene_linenumber is a dict of 20000 dict, each of which has 49 items, then it can saved into 20000 files!
#     filenames_y = glob.glob("/Users/cjr/Documents/Enloc/eQTL/GTEx_Analysis_v8_eQTL_expression_matrices/*.gz")
#     filenames_y.sort()
#     gene_linenumber = []
#     fields = ['gene_id', ]
#     for i in range(len(filenames_y)): # create 49 `gene:line_number`
#         gene_linenumber.append({})

#     for i in range(len(filenames_y)): # construct each of the 49 `gene:line_number`
#         gene_id = pd.read_csv(filenames_y[i], sep='\t', skipinitialspace=True, usecols=fields)
#         for j in range(gene_id.shape[0]): # go over lines
#             gene_linenumber[i][gene_id['gene_id'].tolist()[j]] = j + 2
#             print('%s at line %d in file %d' % (gene_id['gene_id'].tolist()[j], j + 2, i))

    # stupid! why not directly go instead of search over ensembl_gene_id?!
#     with open("/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/ensembl_gene_id.txt") as f:
#         ensembl_gene_id = [line.rstrip("\n") for line in f]

#     filenames_y = glob.glob("/project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_expression_matrices/*.gz")
#     gene_linenumber = []
#     fields = ['gene_id', ]
#     for i in range(len(filenames_y)): # create 49 dict of `gene:line_number`
#         gene_linenumber.append({})
    
#     for i in range(len(filenames_y)): # construct each of the 49 `gene:line_number`
#         gene_id = pd.read_csv(filenames_y[i], sep='\t', skipinitialspace=True, usecols=fields)
#         for j in range(len(ensembl_gene_id)): # search over ensembl_gene_id (turns out to be extremely stupid!)
#             if (gene_id.loc[gene_id['gene_id'] == ensembl_gene_id[j]].shape[0] == 0):
#                 continue
#             number = gene_id.loc[gene_id['gene_id'] == ensembl_gene_id[j]].index[0] + 2
#             gene_linenumber[i][ensembl_gene_id[j]] = number
#             print('found ensembl_gene_id[%d] = %s in line %d of file %d' % (j, ensembl_gene_id[j], number, i))
    

    # the following code is abandoned! Remember Gao Wang's words!!!
#     gene_linenumber = {} # `gene:line_number`
#     import gzip
#     for i in range(len(ensembl_gene_id)): # search over genes
#         with gzip.open("/project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_expression_matrices/Adipose_Subcutaneous.v8.normalized_expression.bed.gz") as f: # construct `gene:line_number` for each tissue respectively
#             for number, line in enumerate(f, 1): # search the gene in the file
#                 if ensembl_gene_id[i].encode() in line: # once match, record it into `gene:line_number` and proceed to the next gene
#                     print('found ensembl_gene_id[%d] = %s in line %d.' % (i, ensembl_gene_id[i], number))
#                     gene_linenumber[ensembl_gene_id[i]] = number
#                     break
    
R: expand = True
    # `gene_TSS.csv`
    ensembl_gene_id <- fread(file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/ensembl_gene_id.txt", sep = "\n", quote = "", header = FALSE)
    gene_TSS <- getBM(attributes = c("chromosome_name", "transcript_start", "ensembl_gene_id", "ensembl_gene_id_version"), filters = "ensembl_gene_id_version", values = ensembl_gene_id, mart = mart)
    rm(ensembl_gene_id)
    write.table(x = gene_TSS, file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/gene_TSS.csv", sep = '\t', quote = FALSE, col.names = TRUE, row.names = FALSE)
    # gene_TSS_retrieval <- read.table(file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/gene_TSS.csv", sep = '\t', quote = "", stringsAsFactors = FALSE, header = TRUE)
    
    # prepare expression of all the 20000 genes in each tissue and the directory for query.
    filenames_y <- list.files(path = "/project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_expression_matrices", pattern = "*.gz$", full.names = TRUE) # expression of 20000 genes in 49 tissues (49 files)
    read_expression <- function(x) {{
        y_allgenes_tmp <- scan(file = x, what = "", sep = "\n")
        y_allgenes <- strsplit(x = y_allgenes_tmp, split = "[[:space:]]+") # The most time-consuming step
        return(y_allgenes)
    }}
    y_total <- lapply(filenames_y, read_expression)
    

In [ ]:
# extract Z, y, X from covariates, expression and genotype data and generate y_res
[extract]
parameter: ensembl_gene_id = str
bash: expand = '${ }'
    # y
#     zless /project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_expression_matrices/*.gz | grep -w "${ensembl_gene_id}" > /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/${ensembl_gene_id}.txt
    
    # X
    awk '$4 ~ /{ensembl_gene_id}/ {{print $0}}' /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/gene_TSS.csv | head -n 1 | awk '{{$3=$2+1000000}} {{$2=$2-1000000}} {{print "chr"$1":"$2"-"$3}}' > /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/{ensembl_gene_id}_TSSregion.txt
    tabix /project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/genotypes/WGS/variant_calls/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.vcf.gz "$(< /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/{ensembl_gene_id}_TSSregion.txt)" | sed -e 's/0|0/0/g' -e 's/0|1/1/g' -e 's/1|0/1/g' -e 's/1|1/2/g' > /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/{ensembl_gene_id}_genotype.txt
    
R: expand = True
    # Z
    filenames <- list.files(path = "/project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_covariates", pattern = "*.txt", full.names = TRUE)
    Z <- lapply(filenames, function(x) t(as.matrix(read.table(file = x, header = TRUE, sep = '\t', quote = "", row.names = 1))))
    for (i in 1:49) {{ # the format of sample names Z was changed somehow, from "GTEX-*" to "GTEX.*", so we need to convert it back.
        rownames(Z[[i]]) <- gsub(pattern = "GTEX.", replacement = "GTEX-", x = rownames(Z[[i]]))
    }}
    
    # y
    
    filenames_y <- list.files(path = "/project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_expression_matrices", pattern = "*.gz$", full.names = TRUE)
    extract_y <- function(i) {{
        yi <- fread(file = filenames_y[i], skip = line_numbers[i,] - 1, nrows = 1)
        samplenames_yi <- fread(file = filenames_y[i], skip = 0, nrows = 1)
        colnames(yi) <- colnames(samplenames_yi)
        yi <- t(as.matrix(yi))
        yi <- yi[-1:-4, , drop = FALSE]
        return(yi)
    }}
    
    y <- lapply(1:49, extract_y)
    
    ## sample names matching between y and Z
    samplenamesmatching <- function(x, reference) {{
        lapply(1:49, function(i) x[[i]][match(rownames(reference[[i]]), rownames(x[[i]])), , drop = FALSE])
    }}
    
    y_matchZ <- samplenamesmatching(x = y, reference = Z)
    y <- y_matchZ
    
    # X
    X <- read.csv(file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/{ensembl_gene_id}_genotype.txt", sep = '\t', header = FALSE, row.names = 3, stringsAsFactors = FALSE)
    X <- X[,-(1:8)]
    samplenames_X <- scan(file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/samplenames_X.txt", what = character(), quote = "")
    samplenames_X <- samplenames_X[-(1:9)]
    colnames(X) <- samplenames_X
    X <- as.matrix(x = X)
    X <- t(X)
    
    # y_res
    y_res <- lapply(1:49, function(i) .lm.fit(x = Z[[i]], y = y[[i]])$residuals)

    # save
    saveRDS(object = list(X = X, y = y, Z = Z, y_res = y_res), file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/Multi_Tissues.{ensembl_gene_id}.RDS")

